### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install langchain-chroma
%pip install -U langchain langchainhub --quiet

## 1. 문서의 내용을 읽는다.

In [3]:
from langchain_community.document_loaders import Docx2txtLoader


loader = Docx2txtLoader('tax.docx')
document = loader.load()

print(len(document))

1


## 2. 문서를 쪼갠다.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap=200,  # 텍스트를 분할할 때, 각 chunk(조각) 사이에 중복되는 부분의 토큰 수
)

document_list = loader.load_and_split(text_splitter=text_splitter) # 문서를 쪼갤 때
len(document_list)

220

## 3.1 임베딩 모델 물러오기

In [5]:
from dotenv import load_dotenv                # OpenAI API Key 사용을 위해 환경변수 등록
from langchain_openai import OpenAIEmbeddings # OpenAI 의 Embedding을 사용

load_dotenv() # 환경변수 불러오기

embeddings = OpenAIEmbeddings(model='text-embedding-3-large') # 기본모델은 002인 예전모델이다. 신규 모델을 사용하기 위해 추가

## 3.2 벡터 DB(Chreoma) 생성 후 저장

In [6]:
from langchain_chroma import Chroma

"""
documents         : 쪼갠 문서
embedding         : 임베딩 모델
collection_name   : 테이블명
persist_directory : DB저장 경로
"""
database = Chroma.from_documents(documents=document_list, embedding=embeddings, collection_name='chroma_tax',persist_directory="./chroma")

## 4. 질문 생성 후 벡터DB에서 유사도검색하여 문서가져오기

In [18]:
query = '연봉 1억원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3); # similarity_search() 함수는 유사도 검색을 위한 함수이다. k 값은 가져올 문서 갯수

## 5.1 LLM 생성

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

## 5.2 프롬프트(질의+문서) 작성
- LLM이 문서를 참고하여 답변할 수 있도록 질의와 문서를 포함한 프롬프트 작성
- 페르소나가 중요하다

In [19]:
# 기존에는 질문만 날렸지만 이제는 질문+문서 를 질의한다. 즉, 프롬프트 작성
# prompt = f"""[Identity]
# - 당신은 현존하는 세계 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요
#
# [Context]
# {retrieved_docs}
#
# Question: {query}
# """


## 5.2 프롬프트를 LLM에 전달

In [14]:
# ai_message = llm.invoke(prompt)

In [15]:
# ai_message.content

'연봉 1억원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세율 체계와 공제 항목들을 고려해야 합니다. 기본적으로, 소득세는 과세표준에 따라 누진세율이 적용됩니다. 2023년 기준으로 한국의 소득세율은 다음과 같습니다:\n\n- 과세표준 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억 5,000만원 이하: 35%\n- 1억 5,000만원 초과: 38%\n\n연봉 1억원인 경우, 기본적인 소득세 계산은 다음과 같이 이루어집니다:\n\n1. 과세표준 계산: 연봉에서 인정되는 각종 공제를 차감하여 과세표준을 산출합니다. 일반적으로 건강보험료, 국민연금, 고용보험료 및 기타 공제를 고려해야 합니다. 여기서는 공제를 고려하지 않고 단순히 계산하겠습니다.\n\n2. 소득세 계산:\n   - 1,200만원까지: 1,200만원 * 6% = 72만원\n   - 1,200만원 초과 ~ 4,600만원: (4,600만원 - 1,200만원) * 15% = 510만원\n   - 4,600만원 초과 ~ 8,800만원: (8,800만원 - 4,600만원) * 24% = 1,008만원\n   - 8,800만원 초과 ~ 1억원: (1억원 - 8,800만원) * 35% = 420만원\n\n3. 총 소득세: 72만원 + 510만원 + 1,008만원 + 420만원 = 2,010만원\n\n이 계산은 기본적인 소득세만을 고려한 것으로, 실제 납부해야 할 세금은 공제 항목과 세액공제 등을 통해 더 낮아질 수 있습니다. 정확한 소득세를 계산하기 위해서는 개인의 구체적인 소득 및 공제 정보를 반영해야 합니다.'

### RetrievalQA Chain 사용
- LangChain Hub 사용

In [16]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### QA Chain 사용

In [20]:
# QA Chain 만들기
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(), # as_retriever 는 VetcotrDB에 다 사용가능한 메서드.
    chain_type_kwargs={"prompt":prompt}
)

In [21]:
ai_message = qa_chain({"query" : query})

C:\Users\DataLogics\AppData\Local\Temp\ipykernel_31824\914447175.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query" : query})


In [22]:
ai_message

{'query': '연봉 1억원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 1억원인 직장인의 소득세는 근로소득에 대한 기본세율이 적용됩니다. 기본세율은 소득 구간에 따라 차등 적용되며, 구체적인 금액은 세율표를 통해 계산해야 합니다. 제공된 정보에서 구체적인 세율이 없으므로, 정확한 소득세를 계산하기 위해서는 추가적인 세율표가 필요합니다.'}